In [153]:
import pandas as pd

data = pd.read_json('data/original.json',orient='column')

In [154]:
data['data'][0]['paragraphs'][0]['context']


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [155]:
data['data'][0]['paragraphs'][0]['qas']

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}],
  'id': '5733be284776f41900661182',
  'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
 {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}],
  'id': '5733be284776f4190066117f',
  'question': 'What is in front of the Notre Dame Main Building?'},
 {'answers': [{'answer_start': 279, 'text': 'the Main Building'}],
  'id': '5733be284776f41900661180',
  'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?'},
 {'answers': [{'answer_start': 381,
    'text': 'a Marian place of prayer and reflection'}],
  'id': '5733be284776f41900661181',
  'question': 'What is the Grotto at Notre Dame?'},
 {'answers': [{'answer_start': 92,
    'text': 'a golden statue of the Virgin Mary'}],
  'id': '5733be284776f4190066117e',
  'question': 'What sits on top of the Main Building at Notre Dame?'}]

In [156]:
data['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']

'Saint Bernadette Soubirous'

In [157]:
# for each data['data'][0]
data['data'][0]['paragraphs'][1]['context']

"As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a li

In [171]:
#file = open('data/paragraphs.txt','w') 
 
#file.writelines(paragraphs) 
#file.close()

In [176]:
#All paragraphs
paragraphs = []
answers = []

for articleIndex in range(data['data'].size):
    for paragraphIndex in range (len(data['data'][articleIndex]['paragraphs'])):
        paragraphs.append(data['data'][articleIndex]['paragraphs'][paragraphIndex]['context'])
        
        #Gather all answers for the paragraph
        paragraphAnswers = []
        for QAIndex in range(len(data['data'][articleIndex]['paragraphs'][paragraphIndex]['qas'])):
            paragraphAnswers.append(data['data'][articleIndex]['paragraphs'][paragraphIndex]['qas'][QAIndex]['answers'][0]['text'])

        #Gather answers for paragrapgh with the others
        answers.append(paragraphAnswers)
        
print (len(paragraphs))
print (len(answers))

18896
18896


In [178]:
paragraphs[1]

"As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a li

Let's clean the data. I'm guessing that the mechanical turks have been imaginative and have used different words, not exactly as in the text, for the answers.
Probably there are a few additional words like "a" and "the" or some synonyms used which I can clean. 

In [179]:
answerInText = []

for paragraphIndex in range(len(paragraphs)):
    paragraphAnswers = []
    for answerIndex in range(len(answers[paragraphIndex])):
        paragraphAnswers.append(answers[paragraphIndex][answerIndex] in paragraphs[paragraphIndex])
    answerInText.append(paragraphAnswers)
    
print ('answers in text: ' + str(sum(x.count(True) for x in answerInText)))
print ('answers not in text: ' + str(sum(x.count(False) for x in answerInText)))

answers in text: 87599
answers not in text: 0


In [180]:
paragraphs[24]

'The Rev. Theodore Hesburgh, C.S.C., (1917–2015) served as president for 35 years (1952–87) of dramatic transformations. In that time the annual operating budget rose by a factor of 18 from $9.7 million to $176.6 million, and the endowment by a factor of 40 from $9 million to $350 million, and research funding by a factor of 20 from $735,000 to $15 million. Enrollment nearly doubled from 4,979 to 9,600, faculty more than doubled 389 to 950, and degrees awarded annually doubled from 1,212 to 2,500.'

In [181]:
answers[24]

['1917–2015', '18', '$9 million', '1952–87', '950']

Really. Absolutely no imagination?

Let's lowercase all text.

I also see a lot of numbers. But different numbers would be harder for the neural network to catch. Let's encode all the numbers to be more similar.If we switch each digit with a '#'we'll also preserve the different kinds of numbers. #### would most likely represent years. ## - age and so on.

The same could be done for names. Especially if the entire article is for a person or a place, like Gengis Khan, we can encode that as well.

In [182]:
import re

for paragraphIndex in range(len(paragraphs)):
    paragraphs[paragraphIndex] = re.sub('\d', '#', paragraphs[paragraphIndex])
    paragraphs[paragraphIndex] = paragraphs[paragraphIndex].lower()
    
for paragraphAnswersIndex in range(len(answers)):
    for answerIndex in range(len(answers[paragraphAnswersIndex])):
        answers[paragraphAnswersIndex][answerIndex] = re.sub('\d', '#', answers[paragraphAnswersIndex][answerIndex])
        answers[paragraphAnswersIndex][answerIndex] = answers[paragraphAnswersIndex][answerIndex].lower()


In [183]:
paragraphs[424]

'the first generation ipod nano may overheat and pose a health and safety risk. affected ipod nanos were sold between september #### and december ####. this is due to a flawed battery used by apple from a single battery manufacturer. apple recommended that owners of affected ipod nanos stop using them. under an apple product replacement program, affected nanos were replaced with current generation nanos free of charge.'

In [184]:
answers[24]

['####–####', '##', '$# million', '####–##', '###']

Should I 
(1) treat each answer paired with its paragraph as a sample or should 
(2) each paragraph be a sample with all its answers concatenated. 
(2) would potentially give me more answer worthy phrases from a text, but it would be harder to train. (1) Would give me only one answer for a large paragraph where there may be many answer worthy phrases. I think (1) is the better solution.

In [185]:
paragraphSamples = []
answerSamples = [];

for paragraphAnswersIndex in range(len(answers)):
    for answerIndex in range(len(answers[paragraphAnswersIndex])):
        paragraphSamples.append(paragraphs[paragraphAnswersIndex])
        answerSamples.append(answers[paragraphAnswersIndex][answerIndex])

print (len(paragraphSamples))
print (len(answerSamples))

87599
87599


Before we split the data into train, validation, test we need to shuffle.

In [186]:
import random

combined = list(zip(paragraphSamples, answerSamples))
random.shuffle(combined)

paragraphSamples[:], answerSamples[:] = zip(*combined)

Let's check if everything is still in order.

In [187]:
inText = []

for index in range(len(paragraphSamples)):
    inText.append(answerSamples[index] in paragraphSamples[index])
    
print ('answers in text: ' + str(inText.count(True)))
print ('answers not in text: ' + str(inText.count(False)))

answers in text: 87599
answers not in text: 0


Let's split our data into text files to be passed on to the NN.

In [234]:
tenthSize = int(len(paragraphSamples) / 10)

#Test set
test_text = open('data/test_text.txt', 'w')
test_answer = open('data/test_answer.txt', 'w')

for index in range(tenthSize):
    test_text.write("%s\n" % paragraphSamples[index].encode("utf-8"))
    test_answer.write("%s\n" % answerSamples[index].encode("utf-8"))
    
test_text.close()
test_answer.close()

#Validation set
val_text = open('data/val_text.txt', 'w')
val_answer = open('data/val_answer.txt', 'w')
    
for index in range(index, tenthSize * 3):
    val_text.write("%s\n" % paragraphSamples[index].encode("utf-8"))
    val_answer.write("%s\n" % answerSamples[index].encode("utf-8"))
    
val_text.close()
val_answer.close()

#Train set
train_text = open('data/train_text.txt', 'w')
train_answer = open('data/train_answer.txt', 'w')

for index in range(index, len(paragraphSamples)):
    train_text.write("%s\n" % paragraphSamples[index].encode("utf-8"))
    train_answer.write("%s\n" % answerSamples[index].encode("utf-8"))
    
train_text.close()
train_answer.close()